In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, load_metric
from sklearn.model_selection import train_test_split
from transformers import pipeline
import tweepy
from flair.embeddings import TransformerDocumentEmbeddings
from flair.data import Sentence
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
import nltk
from utils import numerical_df
from baselines import xgboost_baseline, majority_class_baseline, random_class_baseline
from sklearn.metrics import f1_score as f1, precision_score as ps, recall_score as rs
from sklearn.model_selection import KFold

### Hyperparameters

In [2]:

df = numerical_df(pd.read_csv('full_development_set.csv'))
## Remove empty strings
df = df[df.tweet != '']
df = df[df.tweet.notnull()]

In [3]:
majority_class_baseline(df)

,Task,F1,Precision,Recall
0,argumentative,0.6166666666666667,0.6166666666666667,0.6166666666666667
1,claim,0.675,0.675,0.675
2,evidence,0.8958333333333334,0.8958333333333334,0.8958333333333334
3,procon,0.6142857142857142,0.6142857142857142,0.6142857142857142


In [4]:
random_class_baseline(df)

,Task,F1,Precision,Recall
0,argumentative,0.5291666666666666,0.5291666666666666,0.5291666666666666
1,claim,0.5333333333333334,0.5333333333333334,0.5333333333333334
2,evidence,0.85,0.85,0.85
3,procon,0.5928571428571429,0.5928571428571429,0.5928571428571429


In [ ]:
#res = xgboost_baseline(df)

In [ ]:
def dnummy_class_baseline(df, tasks = possible_tasks, fold = 10, strategy="stratified"):
    
    if isinstance(tasks, str):
        tasks = [tasks]
    
    if not isinstance(tasks, list):
        return ValueError('Tasks is not a string or a list.')
    
    if not all(item in possible_tasks for item in tasks):
        return ValueError(f'Tasks can only be or contain the following elements {possible_tasks} but found {tasks}')
    
    res = []
    
    for task in tasks:
        kf = KFold(n_splits=fold)
        data = np.array(df[task].values).astype(int) 
        if task == 'procon':
            data = ((data[data != 0] + 1)/2).astype(int)
        tres = [] 
        
        for train_index, test_index in kf.split(data):
            
            dummy_clf = DummyClassifier(strategy=strategy)
            X_train, X_test = data[train_index], data[test_index]  
            dummy_clf.fit(X_train, X_train)

            labels = dummy_clf.predict(X_test)
        
            tres.append((f1(X_test, labels, average='micro'), ps(X_test, labels, average='micro'), rs(X_test, labels, average='micro')))
        res.append(np.concatenate([[task], np.mean(tres, axis=0)]))
    res = pd.DataFrame(res)
    res.columns = ['Task', 'F1', 'Precision', 'Recall']
    res = res.style.set_caption('Majority class results')
    return res

In [ ]:
random_class_baseline(df, strategy = 'prior')

In [ ]:
majority_class_baseline(df)

In [ ]:
ibm_baseline(df, 'evidence')

In [ ]:
ibm_baseline(df, 'procon')